<a href="https://colab.research.google.com/github/sifatbhuiyan0909/Dhaka-Finance-Navigator/blob/main/notebooks/08_feature_engineering_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files
uploaded= files.upload()

Saving Dhaka-Stock-Exchange-DSE-2021.csv to Dhaka-Stock-Exchange-DSE-2021.csv
Saving Dhaka-Stock-Exchange-DSE-2020.csv to Dhaka-Stock-Exchange-DSE-2020.csv
Saving Dhaka-Stock-Exchange-DSE-2019.csv to Dhaka-Stock-Exchange-DSE-2019.csv
Saving Dhaka-Stock-Exchange-DSE-2018.csv to Dhaka-Stock-Exchange-DSE-2018.csv
Saving Dhaka-Stock-Exchange-DSE-2017.csv to Dhaka-Stock-Exchange-DSE-2017.csv
Saving Dhaka-Stock-Exchange-DSE-2016.csv to Dhaka-Stock-Exchange-DSE-2016.csv
Saving Dhaka-Stock-Exchange-DSE-2015.csv to Dhaka-Stock-Exchange-DSE-2015.csv
Saving Dhaka-Stock-Exchange-DSE-2014.csv to Dhaka-Stock-Exchange-DSE-2014.csv
Saving Dhaka-Stock-Exchange-DSE-2013.csv to Dhaka-Stock-Exchange-DSE-2013.csv
Saving Dhaka-Stock-Exchange-DSE-2012.csv to Dhaka-Stock-Exchange-DSE-2012.csv
Saving Dhaka-Stock-Exchange-DSE-2011.csv to Dhaka-Stock-Exchange-DSE-2011.csv
Saving Dhaka-Stock-Exchange-DSE-2010.csv to Dhaka-Stock-Exchange-DSE-2010.csv
Saving Dhaka-Stock-Exchange-DSE-2009.csv to Dhaka-Stock-Exchange

In [19]:
import pandas as pd
import glob
import os
import numpy as np
import pandas_ta as ta # Crucial library for technical indicators

# --- 1. Day 3: Data Acquisition and Merging ---
print("--- Day 3: Merging Data ---")
all_filenames = glob.glob(os.path.join('.', '*.csv'))
master_df = pd.concat([pd.read_csv(f, header=None) for f in all_filenames], ignore_index=True)
master_df.columns = ['Ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume']
df = master_df.copy()

# --- 2. Day 4: Structuring, Cleaning Types, and Fixing Duplicates ---
print("\n--- Day 4: Structuring and Cleaning Types ---")
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')

# Clean and convert numerical columns
for col in ['Open', 'Low', 'Volume']:
    df[col] = df[col].astype(str).str.replace('-', '', regex=False)
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Critical Fix: Drop duplicate Date/Ticker entries.
df = df.drop_duplicates(subset=['Date', 'Ticker'], keep='first')
df = df.set_index('Date')
print(f"Duplicates dropped. Data size: {df.shape}")

# --- 3. Day 5: Missing Data Imputation ---
print("\n--- Day 5: Missing Data Imputation ---")
cols_to_fill = ['Open', 'High', 'Low', 'Close', 'Volume']
df[cols_to_fill] = df[cols_to_fill].ffill()

# --- 4. Day 6: Outlier Detection and Smoothing ---
print("\n--- Day 6: Outlier Correction ---")
df['Daily_Return'] = df['Close'].pct_change()
mu = df['Daily_Return'].mean()
sigma = df['Daily_Return'].std()
outlier_mask = (df['Daily_Return'] < mu - 3 * sigma) | (df['Daily_Return'] > mu + 3 * sigma)
df.loc[outlier_mask, ['Open', 'High', 'Low', 'Close', 'Volume']] = None
df[['Open', 'High', 'Low', 'Close', 'Volume']] = df[['Open', 'High', 'Low', 'Close', 'Volume']].ffill()
df = df.drop(columns=['Daily_Return'])

# --- 5. Day 7: Base Feature Engineering ---
print("\n--- Day 7: Base Feature Engineering ---")
df['Log_Return'] = np.log(df['Close'] / df['Close'].shift(1))
df['SMA_5'] = df['Close'].rolling(window=5).mean()
df['SMA_20'] = df['Close'].rolling(window=20).mean()

# --- 6. Day 8: Advanced Feature Engineering (Indicators & Lag) - USING GROUPBY FIX ---
print("\n--- Day 8: Advanced Feature Engineering (Indicators & Lag) ---")

# Prepare for Groupby: Reset index so Ticker is a column we can group by.
df = df.reset_index()

# Define the function to apply all final calculations to a single Ticker's data
def create_advanced_features(group):

    # Calculate RSI
    group['RSI'] = ta.rsi(close=group['Close'], length=14)

    # Calculate MACD
    macd_result = ta.macd(close=group['Close']) # NOTE: Removed append=True

    # --- CRITICAL FIX FOR AttributeError: 'NoneType' ---
    # We must check if the MACD calculation succeeded before proceeding.
    if macd_result is not None:
        # Check for the primary MACD line, falling back if naming convention is different
        if 'MACD_12_26_9' in macd_result.columns:
            group['MACD'] = macd_result['MACD_12_26_9']
        else:
            # Fallback: selects the first column which is usually the main MACD line
            group['MACD'] = macd_result.iloc[:, 0]
    else:
        # If MACD calculation failed (not enough data), fill with NaN to drop later
        group['MACD'] = np.nan

    # Lagged Log Return
    group['Lag_Log_Return'] = group['Log_Return'].shift(1)

    # Drop NaNs created by indicators/lagging *within this group*
    return group.dropna()

--- Day 3: Merging Data ---

--- Day 4: Structuring and Cleaning Types ---
Duplicates dropped. Data size: (1198083, 6)

--- Day 5: Missing Data Imputation ---

--- Day 6: Outlier Correction ---

--- Day 7: Base Feature Engineering ---

--- Day 8: Advanced Feature Engineering (Indicators & Lag) ---
